In [1]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

<ipython-input-1-89e3ae8eb04c>:4: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
<ipython-input-1-89e3ae8eb04c>:5: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc


In [ ]:
# spacex_df 
# CCAFS LC-40 
# VAFB SLC-4E 
# KSC LC-39A
# CCAFS SLC-40

# filtered_df = spacex_df[spacex_df['Launch Site'] == 'CCAFS SLC-40']
# filtered_df

# filtered_ls = filtered_df.groupby(['Launch Site', 'class']).size().reset_index(name='class count')
# filtered_ls

In [2]:
# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
                                
                                dcc.Dropdown(id = 'site-dropdown',
                                             options = [
                                                 {'label': 'All Sites', 'value': 'All'},
                                                 {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                                 {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                                                 {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                                 {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'}
                                             ],
                                                 value = 'All',
                                                 placeholder = "place holder here",
                                                 searchable = True
                                ),                                
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id = 'success-pie-chart')),
                                html.Br(),
                                html.P("Payload range (Kg):"),
                                                                
                                
                                # TASK 3: Add a slider to select payload range
                                #dcc.RangeSlider(id='payload-slider',...)
                                dcc.RangeSlider(id = 'payload-slider', min = 0, max = 10000, step = 1000,
                                #marks = {0: '0',  100: '100'},
                                value = [min_payload, max_payload]),
                                

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id = 'success-payload-scatter-chart')),
                                ])


# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(Output(component_id = 'success-pie-chart', component_property = 'figure'),
              Input(component_id = 'site-dropdown', component_property = 'value'))

def pie(site_dropdown):
    if site_dropdown == 'All':
        title_pie = "Total Success Launches for All sites" 
        fig = px.pie(spacex_df, values='class', names = 'Launch Site', title=title_pie)
        return fig
    else:
        filtered_df = spacex_df[spacex_df['Launch Site'] == site_dropdown]
        filtered_ls = filtered_df.groupby(['Launch Site', 'class']).size().reset_index(name = 'class count')
        title_pie = f"Total Success Launches for site: {site_dropdown}"
        fig = px.pie(filtered_ls, values = 'class count', names = 'class', title = title_pie)
        return fig


# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id = 'success-payload-scatter-chart', component_property = 'figure'), 
              [Input(component_id = 'site-dropdown', component_property = 'value'),
              Input(component_id = "payload-slider", component_property = "value")])

def scatter(site_dropdown, payload_slider):
    low, high = payload_slider
    dropdown_scatter = spacex_df[(spacex_df['Payload Mass (kg)'] > low) & (spacex_df['Payload Mass (kg)'] < high)]
    if site_dropdown == 'All':
        title_scatter = f"Correlation between Payload and booster version for All sites"
        fig = px.scatter(dropdown_scatter, x = 'Payload Mass (kg)', y = 'class', color = 'Booster Version Category', title = title_scatter)
        return fig
    else:
        dropdown_scatter = dropdown_scatter[dropdown_scatter['Launch site'] == site_dropdown]
        title_scatter = f"Correlation between Payload and Booster Versions for site: {site_dropdown}"
        fig = px.scatter(dropdown_scatter, x = 'Payload Mass (kg)', y = 'class', color = 'Booster Version Category', title = title_scatter)
        return fig




In [3]:
# Run the app
if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [29/Sep/2021 19:43:18] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/Sep/2021 19:43:18] "GET /_dash-component-suites/dash/deps/polyfill@7.v2_0_0m1632472781.12.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [29/Sep/2021 19:43:18] "GET /_dash-component-suites/dash/deps/react@16.v2_0_0m1632472781.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [29/Sep/2021 19:43:18] "GET /_dash-component-suites/dash/deps/react-dom@16.v2_0_0m1632472781.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [29/Sep/2021 19:43:18] "GET /_dash-component-suites/dash/deps/prop-types@15.v2_0_0m1632472781.7.2.min.js HTTP/1.1" 200 -
127.0.0.1 - - [29/Sep/2021 19:43:18] "GET /_dash-component-suites/dash/dash-renderer/build/dash_renderer.v2_0_0m1632472780.min.js HTTP/1.1" 200 -
127.0.0.1 - - [29/Sep/2021 19:43:18] "GET /_dash-component-suites/dash/dcc/dash_core_components.v2_0_0m1632472781.js HTTP/1.1" 200 -
127.0.0.1 - - [29/Sep/2021 19:43:18] "GET /_dash-component-suites/d

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\Admin\anaconda3\lib\site-packages\pandas\core\indexes\base.py", line 2646, in get_loc
    return self._engine.get_loc(key)
  File "pandas\_libs\index.pyx", line 111, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\index.pyx", line 138, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\hashtable_class_helper.pxi", line 1619, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas\_libs\hashtable_class_helper.pxi", line 1627, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 'Launch site'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Admin\anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\Admin\anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handl

127.0.0.1 - - [29/Sep/2021 19:43:23] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [29/Sep/2021 19:43:26] "POST /_dash-update-component HTTP/1.1" 200 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\Admin\anaconda3\lib\site-packages\pandas\core\indexes\base.py", line 2646, in get_loc
    return self._engine.get_loc(key)
  File "pandas\_libs\index.pyx", line 111, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\index.pyx", line 138, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\hashtable_class_helper.pxi", line 1619, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas\_libs\hashtable_class_helper.pxi", line 1627, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 'Launch site'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Admin\anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\Admin\anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handl

...

127.0.0.1 - - [29/Sep/2021 19:43:30] "POST /_dash-update-component HTTP/1.1" 500 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\Admin\anaconda3\lib\site-packages\pandas\core\indexes\base.py", line 2646, in get_loc
    return self._engine.get_loc(key)
  File "pandas\_libs\index.pyx", line 111, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\index.pyx", line 138, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\hashtable_class_helper.pxi", line 1619, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas\_libs\hashtable_class_helper.pxi", line 1627, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 'Launch site'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Admin\anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\Admin\anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handl

127.0.0.1 - - [29/Sep/2021 19:43:32] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [29/Sep/2021 19:43:39] "POST /_dash-update-component HTTP/1.1" 200 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\Admin\anaconda3\lib\site-packages\pandas\core\indexes\base.py", line 2646, in get_loc
    return self._engine.get_loc(key)
  File "pandas\_libs\index.pyx", line 111, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\index.pyx", line 138, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\hashtable_class_helper.pxi", line 1619, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas\_libs\hashtable_class_helper.pxi", line 1627, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 'Launch site'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Admin\anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\Admin\anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handl

127.0.0.1 - - [29/Sep/2021 19:43:39] "POST /_dash-update-component HTTP/1.1" 500 -
